# Generate Word Frequency in Audit Reports

## Steps Taken

1. Download the folder with audit reports from OneDrive
2. Iterate through the folder and get the folders with each country's reports
3. For each country's folder, get the country's reports
4. Lemmatize the words in each report
5. Filter the words to remove noise
6. Get the frequency distribution of each text
7. Enter the frequency distribution into a pandas dataframe
8. Merge each country's frequency distributions into one dataframe with the words as rows and report name as column
9. Enter each country as a spreadsheet in an excel workbook
10. Delete the downloaded documents

In [1]:
# link to audit_reports; publicly accessible
audit_reports_link = "https://stir-my.sharepoint.com/:f:/g/personal/fkc3_stir_ac_uk/Esgp-VMQyzBClY5vpTP9TsYBTCb16iA3NvelLEJM53VEgQ?e=7ejaR3"

### 2. Iterate through the folder and create a dictionary to match each country to files within it

In [2]:
import os
from pdfminer.high_level import extract_text
import nltk
import pandas as pd

In [3]:
audit_reports_file_path = "./audit_reports"

In [4]:
country_audit_report_dict = {}

for root, dirs, files in os.walk(audit_reports_file_path):
    if (dirs): # ignore the country folders as children
        continue
    country_audit_report_dict[root] = files;

### 3. For each country's folder, get the country's reports

In [5]:
country_texts_dict = {}

for country_folder_path, filenames in country_audit_report_dict.items():
    for filename in filenames:
        print(f'Doing {country_folder_path}/{filename}')
        try:
            text = extract_text(f'{country_folder_path}/{filename}').lower()
        
            if text and country_folder_path in country_texts_dict:
                country_texts_dict[country_folder_path][filename] = text;
            elif text and country_folder_path not in country_texts_dict:
                country_texts_dict[country_folder_path] = {filename: text}
        except:
            print(f'{filename} has a problem')


Doing ./audit_reports/Zambia/ZAM_GenGov_2015.pdf
Doing ./audit_reports/Zambia/ZAM_LocGov_2021.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2008.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2012.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2003.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2005.pdf
Doing ./audit_reports/Zambia/ZAM_LocGov_2022.pdf
Doing ./audit_reports/Zambia/ZAM_CDF_2022.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2022.pdf
Doing ./audit_reports/Zambia/ZAM_LocGov_2019.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2021.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2013.pdf
Doing ./audit_reports/Zambia/ZAM_Simplified_2018_19.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2009.pdf
Doing ./audit_reports/Zambia/ZAM_LocGov_2015_16_17.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2007.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2010.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2019.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2006.pdf
Doing ./audit_reports/Zambia/ZAM_GenGov_2020.pdf
Doing ./au

### 4. Lemmatize the words in each report

In [6]:
lemmatizer = nltk.WordNetLemmatizer()

for country_path in country_texts_dict.keys():
    for report_name, report_text in country_texts_dict[country_path].items():
        if report_text:
            words = nltk.word_tokenize(report_text.lower())            
            words = [lemmatizer.lemmatize(word) for word in words]

            country_texts_dict[country_path][report_name] = words

### 5. Filter the words to remove noise

In [7]:
# remove words made up numbers, symbols, less than four characters
def is_ideal_word(word):
    if word.isalpha() and len(word) > 3:
        return True
    return False

    
for country_path in country_texts_dict.keys():
    for report_name, words in country_texts_dict[country_path].items():
        words = list(filter(is_ideal_word, words))
        country_texts_dict[country_path][report_name] = words

### 6. Get the frequency distribution of each text

In [8]:
country_report_freq_dist_dict = {}

for country_path in country_texts_dict.keys():
    for report_name, words in country_texts_dict[country_path].items():
        fdist = nltk.FreqDist(words)

        if country_path in country_report_freq_dist_dict:
            country_report_freq_dist_dict[country_path][report_name] = fdist
        else:
            country_report_freq_dist_dict[country_path] = {report_name: fdist}

### 7. Enter the frequency distributions into a pandas dataframe

In [9]:
country_report_df_dict = {}

for country_path in country_report_freq_dist_dict.keys():
    for report_name, freq_dist in country_report_freq_dist_dict[country_path].items():
        df = pd.DataFrame.from_dict(dict(freq_dist), orient='index', columns=[report_name])

        if country_path in country_report_df_dict:
            country_report_df_dict[country_path][report_name] = df
        else:
            country_report_df_dict[country_path] = {report_name: df}

### 8. Merge each country's frequency distributions into one dataframe with the words as rows and report name as column

In [20]:
country_df_dict = {}

for country_path in country_report_df_dict.keys():
    df = pd.DataFrame()
    for report_df in country_report_df_dict[country_path].values():
        if df.empty:
            df = report_df
        else:
            df = pd.merge(df, report_df, 'outer',left_index=True, right_index=True)
    country_df_dict[country_path] = df

In [31]:
country_df_dict['./audit_reports/Nigeria'][:100]

,NIG_congov_2011.pdf,NIG_congov_2019.pdf,NIG_congov_2013.pdf,NIG_congov_2020.pdf,NIG_congov_2007.pdf,NIG_congov_2008.pdf,NIG_congov_2018.pdf
ababa,NaN,12.0,1.0,NaN,NaN,NaN,NaN
abacha,NaN,NaN,3.0,NaN,NaN,NaN,NaN
abagana,NaN,13.0,NaN,NaN,NaN,NaN,NaN
abaji,7.0,13.0,5.0,NaN,NaN,5.0,NaN
abak,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
accompany,1.0,NaN,1.0,NaN,1.0,NaN,NaN
accompanying,1.0,2.0,1.0,NaN,NaN,1.0,NaN
accomplished,NaN,NaN,1.0,NaN,1.0,NaN,NaN
accomplishment,1.0,NaN,NaN,NaN,1.0,NaN,NaN


In [29]:
country_df_dict['./audit_reports/Malawi'].loc['transparency']

MWI_GenGov_2016.pdf    2.0
MWI_GenGov_2014.pdf    2.0
MWI_LocGov_2012.pdf    5.0
MWI_LocGov_2020.pdf    2.0
MWI_GenGov_2007.pdf    NaN
MWI_LocGov_2016.pdf    2.0
MWI_GenGov_2008.pdf    2.0
MWI_GenGov_2020.pdf    7.0
MWI_LocGov_2017.pdf    NaN
MWI_GenGov_2009.pdf    1.0
MWI_GenGov_2021.pdf    2.0
MWI_LocGov_2018.pdf    8.0
MWI_LocGov_2019.pdf    7.0
MWI_GenGov_2022.pdf    7.0
MWI_GenGov_2013.pdf    5.0
MWI_GenGov_2006.pdf    NaN
MWI_GenGov_2017.pdf    7.0
MWI_GenGov_2010.pdf    3.0
MWI_GenGov_2012.pdf    6.0
MWI_GenGov_2019.pdf    6.0
MWI_GenGov_2018.pdf    2.0
MWI_LocGov_2014.pdf    4.0
MWI_GenGov_2011.pdf    1.0
MWI_GenGov_2015.pdf    1.0
Name: transparency, dtype: object

In [10]:
d = {'report': 42, 'auditor': 143, 'general': 185}
p = pd.DataFrame.from_dict(d, orient='index', columns=['2022'])
p

,2022
report,42
auditor,143
general,185


In [11]:
zam_2015 = country_report_df_dict['./audit_reports/Zambia']['ZAM_GenGov_2015.pdf']
zam_2015

,ZAM_GenGov_2015.pdf
report,142
auditor,42
general,185
account,185
republic,17
...,...
examining,1
concealing,1
doing,1
derived,1
